In [ ]:
%matplotlib widget
import numpy as np
from matplotlib import pyplot as plt
from diffraction_simulation import diffraction_simulation as dsim

dsim.StructureHandler.LENGTH_UNIT = 'angstrom'

ewald = dsim.EwaldSphere(300e3)
p21c = dsim.StructureHandler.from_matproj('HfO2_P21c')
pca21 = dsim.StructureHandler.from_matproj('HfO2_Pca21')
p42nmc= dsim.StructureHandler.from_matproj('HfO2_P42nmc').BCT2FCC_transform()
p21c.align_structure([0,0,-1])

fig = plt.figure(figsize=(12,4))
ax1 = fig.add_subplot(131, projection='3d', proj_type='ortho')
p21c.plot_unit_cell_3d(ax1, low_frac=(-0.1,-0.1,-0.01))
ax1.legend()

ax2 = fig.add_subplot(132, projection='3d', proj_type='ortho')
pca21.plot_unit_cell_3d(ax2, high_frac=(1.1, 1.1, 1.05))
ax2.legend()

ax3 = fig.add_subplot(133, projection='3d', proj_type='ortho')
p42nmc.plot_unit_cell_3d(ax3)
ax3.legend()

In [ ]:
from matplotlib.animation import FuncAnimation
fig = plt.figure(figsize=(11,4))
ax1 = fig.add_subplot(131, projection='3d', proj_type='ortho')
p21c.plot_unit_cell_3d(ax1, low_frac=(-0.1,-0.1,-0.01))
# ax1.legend()

ax2 = fig.add_subplot(132, projection='3d', proj_type='ortho')
pca21.plot_unit_cell_3d(ax2, high_frac=(1.1, 1.1, 1.05))
# ax2.legend()

ax3 = fig.add_subplot(133, projection='3d', proj_type='ortho')
p42nmc.plot_unit_cell_3d(ax3)
# ax3.legend()

motion_time = 2
hold_time = 4
frame_time = 0.1
def anim(t):
    if t<motion_time:
        azim = 90*t/motion_time
        elev = 0
    elif t<motion_time+hold_time:
        azim = 90
        elev = 0
    elif t<2*motion_time+hold_time:
        azim = 90
        elev = 90*(t-motion_time-hold_time)/motion_time
    elif t<2*motion_time+2*hold_time:
        azim = 90
        elev = 90
    elif t<=3*motion_time+2*hold_time:
        elev = 90 - 90*(t-2*motion_time-2*hold_time)/motion_time
        azim = 90 - 90*(t-2*motion_time-2*hold_time)/motion_time
    else:
        elev = azim = 0
    for ax in (ax1,ax2,ax3):
        ax.view_init(elev=elev,azim=azim,roll=0)

T = 3*(motion_time+hold_time)
ani = FuncAnimation(fig, anim, 
                    frames=np.arange(0,T,frame_time),
                    interval=int(1000*frame_time), 
                    blit=False) # blit=True for faster rendering
ani.save('./HfO2-similarities.gif', dpi=50)
plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(12,4))
ax1 = fig.add_subplot(131)
pca21.plot_unit_cell_2d(ax1, high_frac=(1.1, 1.1, 1.05), proj_ax_uvw=(0,0,1), x_ax_uvw=(1,0,0))
ax1.set_xlabel('x')
ax1.set_ylabel('y')
# ax1.legend()

ax2 = fig.add_subplot(132)
pca21.plot_unit_cell_2d(ax2, high_frac=(1.1, 1.1, 1.05), proj_ax_uvw=(0,-1,0), x_ax_uvw=(1,0,0))
ax2.set_xlabel('x')
ax2.set_ylabel('z')
# ax2.legend()

ax3 = fig.add_subplot(133)
pca21.plot_unit_cell_2d(ax3, high_frac=(1.1, 1.1, 1.05), proj_ax_uvw=(1,0,0), x_ax_uvw=(0,1,0))
ax3.set_xlabel('y')
ax3.set_ylabel('z')
# ax3.legend()

# Diffraction

In [ ]:
%matplotlib inline
dsim.StructureHandler.LENGTH_UNIT = 'nm'

g_max = 5*np.sqrt(2)
dq = 0.025
sg_max = 0.1

fig,ax = plt.subplots(1,3,figsize=(9,3))
for ax, struct in zip(ax, (p21c, pca21, p42nmc )):
    powder = struct.powder_hkl(ewald, sg_max, g_max, num_orientations=1000)
    powder.pcolor(ax, tight=True, Nq=int(2*g_max/dq/np.sqrt(2)), sample_thickness=10, norm='log', color='gray')
    ax.set_aspect(1)

fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,2))

bins = np.linspace(1.9, g_max, 100)
pca21.  powder_hkl(ewald, sg_max, g_max, num_orientations=10000).hist(ax, sample_thickness=10, bins=bins, color='r', alpha=0.5, label='$Pca2_1$')
p42nmc. powder_hkl(ewald, sg_max, g_max, num_orientations=10000).hist(ax, sample_thickness=10, bins=bins, color='g', alpha=0.5, label='$P4_2/nmc$')
p21c.   powder_hkl(ewald, sg_max, g_max, num_orientations=10000).hist(ax, sample_thickness=10, bins=bins, color='b', alpha=0.5, label='$P2_1/c$')

ax.set_ylabel('Intensity (normalized)')
ax.legend()
ax.minorticks_on()

In [ ]:
dps = []

dps.append(p21c.powder_hkl(ewald, sg_max, g_max, num_orientations=10000))
dps.append(pca21.powder_hkl(ewald, sg_max, g_max, num_orientations=10000))
dps.append(p42nmc.powder_hkl(ewald, sg_max, g_max, num_orientations=10000))

fig,ax = plt.subplots(1,1,figsize=(6,2))
bins = np.linspace(1.9, g_max, 75)
dsim.ElectronDiffractionSpots.stacked_hist(ax, *dps, sample_thickness=10, bins=bins, color=['r','lime','b'], label=['$P2_1/c$','$Pca2_1$','$P4_2/nmc$'])

ax.set_ylabel('Stacked intensity')
ax.legend()
ax.minorticks_on()